# <span style="color:#F72585"><center>Introducción a OpenAI Gym</center></span>

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Alejandria/main/Aprendizaje_Reforzado/Imagenes/Hide_and_seek.jpg" width="700" height="500" align="center"/>
</center> 
</figure>


Fuente: [Hide and seek. Wiki commons](https://commons.wikimedia.org/w/index.php?search=hide+and+seek&title=Special:MediaSearch&go=Go&type=image)

## <span style="color:#4361EE">Introducción</span>

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Alejandria/main/Aprendizaje_Reforzado/Imagenes/multi-agent-policy-architecture.png" width="500" height="400" align="center"/>
</center> 
</figure>


Fuente: [OpenAI-blog](https://openai.com/blog/emergent-tool-use/)

En esta lección se presentan los elementos esenciales de la API de OpenAI Gym. Este ha sido un esfuerzo de OpenAI para poner a nuestra disposición una gran cantidad de ambientes, de muy diversa índole que incluye problema clásicos de control optimal, como el que estudiamos hoy, juegos clásicos y más.

La API es bastante simple y esta desarrollada para que muy rápidamente sea posible interactuar con ella para permitirnos desarrollar nuestros propios modelos de aprendizaje reforzado.

## <span style="color:#4361EE">La API OpenAI Gym</span>

In [ ]:
# instalar gym desde conda
#conda install -c conda-forge gym


<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Alejandria/main/Aprendizaje_Reforzado/Imagenes/agente-ambiente.png" width="600" height="500" align="center"/>
</center> 
</figure>


Fuente: Alvaro Montenegro

La API `OpenAI Gym` tiene una rica colección de ambientes para experimentos de aprendizaje reforzado (AR), usando una interfaz unificada.

La clase principal de Gym es `Env` (de environment). Sus métodos y atributos proveen la información necesaria para poder entrenar agentes que interactúen con el medio ambientes. Las piezas más importante disponibles con *Env* son:

- Un conjunto de `acciones` que se permite sean ejecutadas en el ambiente.
- El tamaño y bordes de las `observaciones` que el ambiente le provee al agente.
- Un método *step* para ejecutar un acción. El método regresa la nueva observación, la `recompensa` y la indicación de si el `episodio` ha terminado (*done*). Este método es la interfaz que permite la comunicación entre el ambiente y el agente. Al agente entrega al ambiente una acción seleccionada y este lo retroalimenta a través de este método.
- Un método *reset* que retorna al ambiente a su *estado inicial* y entrega la primera observación.

### <span style="color:#4CC9F0">El espacio de acciones</span>

Las acciones que puede realizar un agente puede ser discretas, continuas o incluso una combinación de ambas. Por ejemplo en un automóvil manejado de manera autónoma puede ser posible oprimir varios botones a la vez (discreto), oprimir el pedal del acelerador o del freno (continuo) y así. El espacio de acciones contiene o modela todas las posibilidades de acción del agente.

### <span style="color:#4CC9F0">El espacio de observaciones</span>

Las observaciones son piezas de información que el ambiente puede entregar al agente en cualquier momento del tiempo, además de la recompensa. Las observaciones pueden ser muy simples como un arreglo unidimensional de números o  pueden ser  tensores muy complejos, como las imágenes provenientes de varias cámaras del automóvil.

Es claro que acciones y observaciones tienen similaridades, por lo que Gym dispone de una clase abstracta llamada *Space* que permite derivar espacios de acciones y espacios de observaciones. La siguiente imagen muestra una diagrama de la clase *Space*.

Existen otras subclases de *Space*, pero estas son las más comúnmente utilizadas.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Alejandria/main/Aprendizaje_Reforzado/Imagenes/jerarquia_clase_Space_Gym.png" width="600" height="400" align="center"/>
</center>
<center><caption>Jerarquía de la clase Space de OpenAI Gym</caption></center>
</figure>

Fuente: Alvaro Montenegro

La clase abstracta *Space* incluye dos métodos relevantes:

* *sample()*: retorna una muestra aleatoria del espacio.
* *contains(x)*: chequea si el argumento es un elemento del dominio del espacio.

Ambos métodos son abstractos y deben ser implementados en las subclases de *Space*.

* La clase *Discrete* representa un conjunto de ítems mutuamente excluyentes, numerados entre 0 y $n-1$. Su único atributo *n*, es una cuenta de los ítems que la clase describe. Por ejemplo. *Discrete(n=4)* puede representar por ejemplo un espacio de acciones de cuatro direcciones para moverse: *[izquierda, derecha, arriba, abajo]*.

* La clase *Box* representa un tensor *n*-dimensional de números racionales con intervalos *[bajo, alto]*. Por ejemplo puede representar el pedal de un acelerador con un valor simple entre 0.0 y 1.0. En pedal sería codificado en tal caso como *Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32)*. Otro ejemplo puede ser la observación de la pantalla en un juego Atari, la cual es RGB de tamaño 210*160: *Box(low=0, high=255, shape=(210, 160, 3), dtype=np.uint8)*.

* La última subclase de *Space* es *Tuple*, la cual permite combinar varias instancias de *Space* juntas.Esta subclase permite crear espacios de acciones y observaciones de la complejidad que se desee.

Un ejemplo del uso de tupla puede ser el siguiente. Supongamos que deseamos modelar algunos controles de un automóvil. Para empezar la dirección (el ángulo de la dirección, en realidad), el acelerador y el freno puede cambiar en cada instante de tiempo. Por otro lado, podemos tener botones discretos para representar las luces direccionales *(apagadas, izquierda, derecha)* y la bocina *(encendida, apagada)*. Entonces, podemos crear *Tupla(spaces=(Box(low=0.0, high=1.0, shape=(3,), dtype=np.float32), Discrete(3), Discrete(2)
)*.

### <span style="color:#4CC9F0">El ambiente</span>

El ambiente es representado por la clase *Env*. Todo ambiente tiene dos miembros de tipo *Space*: 

* *action_space*: acciones permitidas que entrega el ambiente al agente.
* *observation_space*: observaciones proveídas por el ambiente al agente.

Además el ambiente tiene los siguientes miembros:

* *reset()*: coloca el ambiente en el estado inicial.
* *step()*: Permite al agente tomar una acción y retorna información como respuesta a la acción: la siguiente observación, la recompensa y la bandera indicando si el episodio ha  finalizado. Esta es la pieza central de la funcionalidad del ambiente.

Estas características permiten crear código genérico que podría trabajar con cualquier ambiente.

## <span style="color:#4361EE">Lista de ambientes en OpenAI Gym</span>

In [ ]:
from gym import envs
all_envs = envs.registry.all()
env_ids = [env_spec.id for env_spec in all_envs]
print(*env_ids, sep=', ...,  ')

## <span style="color:blue">Interactuando con la API de OpenAI Gym</span>

### <span style="color:#4CC9F0">Exploración preliminar de los objetos de Gym</span>

In [ ]:
import gym
env = gym.make('CartPole-v1')

obs = env.reset()
obs

La observación es un arreglo de cuatro posiciones conteniendo respectivamente  *coordenada del centro de masas*, *velocidad*, *ángulo con respecto a la plataforma*, y *velocidad angular*.

In [ ]:
env.action_space

El espacio de acciones es de tipo *Discrete* y contiene un atributo que puede representar dos elementos 0 y 1, o izquierda y derecha.

In [ ]:
env.observation_space

El espacio de acciones es de tipo *Box* conteniendo 4 valores entre los límites mostrado aquí.

In [ ]:
action = 0
observation, reward, done, info = env.step(action)
print('obs: ', np.round(observation,4), end='; ')
print('reward: ', reward, end='; ')
print('done: ', done, end='; ')
print('info: ', info)

Con esta sentencia hemos empujado la plataforma a la izquierda (action=0). Como respuesta se ha obtenido un tupla con la siguiente información:

* una nueva observación: [ 0.0253, -0.5594,  0.0178,  0.9062].
* una recompensa de 1.0,
* la bandera *done* con valor *False*, indicando que le episodio aún no termina, y
* información extra acerca del ambiente que es un diccionario, que en este caso está vacío.

In [ ]:
e.observation_space.sample()

Esta es una muestra aleatoria del espacio de observaciones

### <span style="color:#4CC9F0">Interacción con el ambiente de Gym</span>

In [ ]:
import numpy as np
import gym
env = gym.make("CartPole-v1")

# revisemos primero algunos objetos de Gym
print('espacio de acciones: ', env.action_space)
print('espacio de observaciones: ', env.observation_space)
print()

# interactuamos con Gym
observation = env.reset()
print('Comenzamos')
print('obs: ', observation)

for i in range(100):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    print(i, ',', end=' ')
    print('primera observación: ', np.round(observation,4), end='; ')
    print('reward: ', reward, end='; ')
    print('done: ', done, end='; ')
    print('info: ', info)
    if done:
        observation = env.reset()
env.close()
print('Hecho')

## <span style="color:#4361EE">Referencias</span>

1. [OpenAI Gym - Github](https://github.com/openai/gym)
1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Richard S. Sutton, Andrew G. Barto, Reinforcement learning: an introduction, 2nd edition, 2020](http://library.lol/main/6502B74CE247C4CD4D4FB54747AD7C7E)
1. [Praveen Palanisamy - Hands-On Intelligent Agents with OpenAI Gym_ Your Guide to Developing AI Agents Using Deep Reinforcement Learning, 2020](http://library.lol/main/E4FD128CF9B93E0F7A542B053330517A)
1. [Turing Paper 1936](http://www.thocp.net/biographies/papers/turing_oncomputablenumbers_1936.pdf)